In [32]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

In [33]:
# Constants
pace_threshold = 0.5
euclidean_distance_threshold = 200

In [34]:
# Read in Split Timings
df = pd.read_csv('../../data/CCC_1km_splits.csv', encoding = "ISO-8859-1")

In [35]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Year,bib,name_mask,gender,category,rank,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
0,0,NaN,2016,3002,1eeca1e237fb722bee11d6b3a89182cf,Male,Senior Men (23-39),NaN,NaN,8.326923,4.449612,5.405405,4.506410,NaN,10.874074,NaN,4.446667,NaN,6.095238,NaN,NaN,NaN,8.436578,NaN,NaN,6.081633,11.584906,NaN,NaN,5.625000,NaN,12.222944,NaN,8.888889,NaN,NaN,5.193694,NaN,730.066667,7.295569
1,1,NaN,2016,3003,79c472880467010197a2e94fbb718b52,Male,Senior Men (23-39),NaN,NaN,8.193910,4.705426,5.454955,4.487179,NaN,11.651852,NaN,4.666667,NaN,6.975000,NaN,NaN,NaN,8.134218,NaN,NaN,5.173469,11.893082,NaN,NaN,6.383333,NaN,11.344156,NaN,7.828283,NaN,NaN,5.087838,NaN,735.333333,7.284241
2,2,NaN,2016,3012,18896d8a594562fbc883979eb5985bd3,Male,Masters Men 1 (40-49),3.0,NaN,8.485577,4.674419,5.777027,4.897436,NaN,12.962963,NaN,5.046667,NaN,6.615476,NaN,NaN,NaN,8.569322,NaN,NaN,5.125850,11.748428,NaN,NaN,4.858333,NaN,10.699134,NaN,8.000000,NaN,NaN,5.299550,NaN,739.350000,7.340013
3,3,3.0,2016,3005,78612f155eb4cf9201cf5802867d8119,Male,Senior Men (23-39),4.0,NaN,8.700321,4.864341,5.691441,4.891026,NaN,12.211111,NaN,4.886667,NaN,6.463095,NaN,NaN,NaN,8.741888,NaN,NaN,5.489796,12.361635,NaN,NaN,4.972222,NaN,11.023810,NaN,7.363636,NaN,NaN,5.680180,NaN,745.500000,7.381512
4,4,4.0,2016,3015,28fe5c601616ac3c31f9af34d49d6e71,Male,Senior Men (23-39),5.0,NaN,8.860577,4.755814,5.695946,5.317308,NaN,14.270370,NaN,4.818333,NaN,6.453571,NaN,NaN,NaN,8.556047,NaN,NaN,5.996599,12.185535,NaN,NaN,5.230556,NaN,11.562771,NaN,9.090909,NaN,NaN,6.211712,NaN,772.183333,7.786146


In [36]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'bib', 'rank'], axis = 1)

In [37]:
df.shape

(20471, 36)

In [38]:
df.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
0,2016,1eeca1e237fb722bee11d6b3a89182cf,Male,Senior Men (23-39),NaN,8.326923,4.449612,5.405405,4.506410,NaN,10.874074,NaN,4.446667,NaN,6.095238,NaN,NaN,NaN,8.436578,NaN,NaN,6.081633,11.584906,NaN,NaN,5.625000,NaN,12.222944,NaN,8.888889,NaN,NaN,5.193694,NaN,730.066667,7.295569
1,2016,79c472880467010197a2e94fbb718b52,Male,Senior Men (23-39),NaN,8.193910,4.705426,5.454955,4.487179,NaN,11.651852,NaN,4.666667,NaN,6.975000,NaN,NaN,NaN,8.134218,NaN,NaN,5.173469,11.893082,NaN,NaN,6.383333,NaN,11.344156,NaN,7.828283,NaN,NaN,5.087838,NaN,735.333333,7.284241
2,2016,18896d8a594562fbc883979eb5985bd3,Male,Masters Men 1 (40-49),NaN,8.485577,4.674419,5.777027,4.897436,NaN,12.962963,NaN,5.046667,NaN,6.615476,NaN,NaN,NaN,8.569322,NaN,NaN,5.125850,11.748428,NaN,NaN,4.858333,NaN,10.699134,NaN,8.000000,NaN,NaN,5.299550,NaN,739.350000,7.340013
3,2016,78612f155eb4cf9201cf5802867d8119,Male,Senior Men (23-39),NaN,8.700321,4.864341,5.691441,4.891026,NaN,12.211111,NaN,4.886667,NaN,6.463095,NaN,NaN,NaN,8.741888,NaN,NaN,5.489796,12.361635,NaN,NaN,4.972222,NaN,11.023810,NaN,7.363636,NaN,NaN,5.680180,NaN,745.500000,7.381512
4,2016,28fe5c601616ac3c31f9af34d49d6e71,Male,Senior Men (23-39),NaN,8.860577,4.755814,5.695946,5.317308,NaN,14.270370,NaN,4.818333,NaN,6.453571,NaN,NaN,NaN,8.556047,NaN,NaN,5.996599,12.185535,NaN,NaN,5.230556,NaN,11.562771,NaN,9.090909,NaN,NaN,6.211712,NaN,772.183333,7.786146


In [39]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
15085,2009,00037fcf2b73cebbc53964ca34bacbea,Female,Masters Women 1 (40-49),24.758503,NaN,NaN,17.013514,11.695513,NaN,22.948148,NaN,9.698333,NaN,13.434524,NaN,19.037037,NaN,NaN,16.828042,NaN,NaN,25.044025,NaN,NaN,15.080556,NaN,23.573593,NaN,14.328283,NaN,NaN,11.797297,NaN,1530.166667,17.325951
17920,2007,000601620b7a27bf6519506df6a09c6c,Male,Masters Men 1 (40-49),13.292517,NaN,NaN,8.265766,8.003205,NaN,15.240741,47.725225,NaN,NaN,NaN,NaN,15.533670,NaN,NaN,10.198413,NaN,NaN,NaN,NaN,NaN,19.630556,NaN,NaN,8.521164,NaN,NaN,8.739464,NaN,NaN,878.616667,15.515072
8449,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015
18728,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566
8339,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,NaN,17.050314,NaN,NaN,6.077778,NaN,NaN,14.325397,NaN,NaN,9.009579,NaN,NaN,887.266667,10.436679


# UTMB

In [40]:
# Read in Split Timings
df1 = pd.read_csv('../../data/1km_splits_full.csv', encoding = "ISO-8859-1")

In [41]:
df1.head()

,Unnamed: 0,Unnamed: 0.1,Year,bib,name_mask,team,gender,category,rank,nationality,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,0,NaN,2017,4.0,7efc0cb7e70d7cd9da46a58bbf01af66,Salomon,Male,Senior Men (23-39),NaN,FR,5.281863,NaN,NaN,4.375000,NaN,5.443366,6.483539,9.900000,4.980392,8.205502,4.044218,9.433333,5.237037,5.646341,NaN,NaN,10.649660,6.691441,6.487179,11.881481,5.323333,5.707143,NaN,NaN,8.030973,NaN,NaN,6.102041,NaN,8.604167,NaN,7.747126,NaN,NaN,7.788288,NaN,NaN,9.016667,5.236486,1141.900000,NaN,7.012357
1,1,NaN,2017,NaN,1e4ba9a22b3fecb9136ecfcf7f470868,Salomon,Male,Senior Men (23-39),NaN,ES,5.147059,NaN,NaN,4.519737,NaN,5.501618,6.510288,10.100000,4.970588,8.098706,4.047619,9.429167,5.255556,5.743902,NaN,NaN,11.159864,6.887387,6.810897,12.225926,5.376667,5.800000,NaN,NaN,8.781711,NaN,NaN,5.768707,NaN,8.729167,NaN,7.235632,NaN,NaN,7.869369,NaN,NaN,9.300000,5.072072,1156.983333,15.083333,7.097568
2,2,NaN,2017,14.0,0d928f7482046900e19dba217d7befa5,Hoka,Male,Senior Men (23-39),3.0,US,5.544118,NaN,NaN,4.372807,NaN,5.500000,6.981481,10.769697,5.689542,8.812298,4.452381,10.020833,5.629630,6.292683,NaN,NaN,11.411565,6.979730,6.519231,NaN,11.085000,5.870238,NaN,NaN,8.631268,NaN,NaN,5.323129,NaN,9.222222,NaN,7.497126,NaN,NaN,7.918919,NaN,NaN,9.804762,4.772523,1193.000000,51.100000,7.352225
3,3,3.0,2017,7.0,93f20063908ba5d48724dcf31740a52a,Asics,Male,Senior Men (23-39),4.0,FR,5.283088,NaN,NaN,4.379386,NaN,5.441748,6.489712,11.700000,5.741830,9.105178,4.812925,10.220833,5.677778,6.516260,NaN,NaN,12.340136,7.243243,7.243590,13.255556,5.170000,5.839286,NaN,NaN,8.510324,NaN,NaN,6.119048,NaN,9.104167,NaN,7.370690,NaN,NaN,7.585586,NaN,NaN,9.238095,4.808559,1203.650000,61.750000,7.466542
4,4,4.0,2017,NaN,7479fb90beffad048c5f772ba1915b4e,Hoka,Male,Senior Men (23-39),5.0,US,5.145833,NaN,NaN,4.245614,NaN,5.582524,6.615226,9.887879,4.973856,8.249191,3.901361,8.366667,5.162963,6.024390,NaN,NaN,10.870748,7.063063,6.448718,11.896296,5.868333,7.513095,NaN,NaN,12.036873,NaN,NaN,4.996599,NaN,8.180556,NaN,6.422414,NaN,NaN,8.414414,NaN,NaN,10.057143,5.141892,1211.633333,69.733333,7.211069


In [42]:
# Drop unnecessary columns
df1 = df1.drop(['Unnamed: 0', 'Unnamed: 0.1', 'team', 'nationality', 'bib', 'rank', 'gender'], axis = 1)

In [43]:
df1.head()

,Year,name_mask,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2017,7efc0cb7e70d7cd9da46a58bbf01af66,Senior Men (23-39),5.281863,NaN,NaN,4.375000,NaN,5.443366,6.483539,9.900000,4.980392,8.205502,4.044218,9.433333,5.237037,5.646341,NaN,NaN,10.649660,6.691441,6.487179,11.881481,5.323333,5.707143,NaN,NaN,8.030973,NaN,NaN,6.102041,NaN,8.604167,NaN,7.747126,NaN,NaN,7.788288,NaN,NaN,9.016667,5.236486,1141.900000,NaN,7.012357
1,2017,1e4ba9a22b3fecb9136ecfcf7f470868,Senior Men (23-39),5.147059,NaN,NaN,4.519737,NaN,5.501618,6.510288,10.100000,4.970588,8.098706,4.047619,9.429167,5.255556,5.743902,NaN,NaN,11.159864,6.887387,6.810897,12.225926,5.376667,5.800000,NaN,NaN,8.781711,NaN,NaN,5.768707,NaN,8.729167,NaN,7.235632,NaN,NaN,7.869369,NaN,NaN,9.300000,5.072072,1156.983333,15.083333,7.097568
2,2017,0d928f7482046900e19dba217d7befa5,Senior Men (23-39),5.544118,NaN,NaN,4.372807,NaN,5.500000,6.981481,10.769697,5.689542,8.812298,4.452381,10.020833,5.629630,6.292683,NaN,NaN,11.411565,6.979730,6.519231,NaN,11.085000,5.870238,NaN,NaN,8.631268,NaN,NaN,5.323129,NaN,9.222222,NaN,7.497126,NaN,NaN,7.918919,NaN,NaN,9.804762,4.772523,1193.000000,51.100000,7.352225
3,2017,93f20063908ba5d48724dcf31740a52a,Senior Men (23-39),5.283088,NaN,NaN,4.379386,NaN,5.441748,6.489712,11.700000,5.741830,9.105178,4.812925,10.220833,5.677778,6.516260,NaN,NaN,12.340136,7.243243,7.243590,13.255556,5.170000,5.839286,NaN,NaN,8.510324,NaN,NaN,6.119048,NaN,9.104167,NaN,7.370690,NaN,NaN,7.585586,NaN,NaN,9.238095,4.808559,1203.650000,61.750000,7.466542
4,2017,7479fb90beffad048c5f772ba1915b4e,Senior Men (23-39),5.145833,NaN,NaN,4.245614,NaN,5.582524,6.615226,9.887879,4.973856,8.249191,3.901361,8.366667,5.162963,6.024390,NaN,NaN,10.870748,7.063063,6.448718,11.896296,5.868333,7.513095,NaN,NaN,12.036873,NaN,NaN,4.996599,NaN,8.180556,NaN,6.422414,NaN,NaN,8.414414,NaN,NaN,10.057143,5.141892,1211.633333,69.733333,7.211069


In [44]:
# Filter out NaN times
df1 = df1.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df1.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df1.head()

,Year,name_mask,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
11228,2013,0001fff82652ce3818504f26771297a3,Senior Men (23-39),7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557
3623,2016,0002b9e98b5761bb4aa0960616f45ca6,Masters Men 1 (40-49),8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491
5634,2015,000bf78327bdfa8c6298fc9a76921305,Masters Men 1 (40-49),7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
7880,2014,000d01e06e9abfe51cdb08ff2567f125,Senior Men (23-39),7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
15238,2009,00156070bc3ed3fd8b6b87dabd51c2fc,Senior Men (23-39),NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108


# Filter out runners who only ran CCC and UTMB

In [45]:
# Count number of races per runner
#v = df.name_mask.value_counts()
# Get runners who ran more than one race
#multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
#df = df[df.name_mask.isin(multi_race_runners)]

In [46]:
df1.head()

,Year,name_mask,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
11228,2013,0001fff82652ce3818504f26771297a3,Senior Men (23-39),7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557
3623,2016,0002b9e98b5761bb4aa0960616f45ca6,Masters Men 1 (40-49),8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491
5634,2015,000bf78327bdfa8c6298fc9a76921305,Masters Men 1 (40-49),7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
7880,2014,000d01e06e9abfe51cdb08ff2567f125,Senior Men (23-39),7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
15238,2009,00156070bc3ed3fd8b6b87dabd51c2fc,Senior Men (23-39),NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108


# With Personal Best Time

In [47]:
# Get Personal Best
pb = df1[df1.time.groupby(df1.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
#ccc = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [48]:
pb.head()

,name_mask,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
11228,0001fff82652ce3818504f26771297a3,7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557
3623,0002b9e98b5761bb4aa0960616f45ca6,8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491
5634,000bf78327bdfa8c6298fc9a76921305,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
7880,000d01e06e9abfe51cdb08ff2567f125,7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
15238,00156070bc3ed3fd8b6b87dabd51c2fc,NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108


In [49]:
name = pb['name_mask']

In [50]:
name.head()

11228    0001fff82652ce3818504f26771297a3
3623     0002b9e98b5761bb4aa0960616f45ca6
5634     000bf78327bdfa8c6298fc9a76921305
7880     000d01e06e9abfe51cdb08ff2567f125
15238    00156070bc3ed3fd8b6b87dabd51c2fc
Name: name_mask, dtype: object

In [51]:
pb = pb.add_suffix('_pb')

In [52]:
pb = pb.drop(['name_mask_pb'], axis=1)

In [53]:
pb = pb.join(name)

In [54]:
pb

,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,name_mask
11228,7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557,0001fff82652ce3818504f26771297a3
3623,8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491,0002b9e98b5761bb4aa0960616f45ca6
5634,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938,000bf78327bdfa8c6298fc9a76921305
7880,7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371,000d01e06e9abfe51cdb08ff2567f125
15238,NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108,00156070bc3ed3fd8b6b87dabd51c2fc
8786,8.161765,NaN,NaN,8.004386,NaN,10.145631,12.958848,18.939394,10.480392,14.407767,10.850340,20.237500,10.292593,14.182927,NaN,NaN,27.693878,12.916667,15.291667,23.518519,12.421667,13.336905,NaN,NaN,23.163717,NaN,NaN,19.513605,25.518868,NaN,18.911111,NaN,25.900433,NaN,NaN,21.712121,NaN,NaN,13.033784,2627.083333,1415.350000,16.316437,001e405237215c01ebb54168bc0d93ec
15711,NaN,NaN,7.596899,NaN,NaN,9.894822,13.777778,18.745455,10.346405,16.810680,8.343537,19.191667,8.600000,12.126016,NaN,NaN,25.476190,12.045045,12.076923,23.107407,9.490000,12.673810,21.461279,NaN,NaN,16.722222,NaN,NaN,30.515723,NaN,20.141667,NaN,27.378788,NaN,NaN,15.545455,NaN,NaN,12.918919,2523.133333,1229.833333,15.868986,0027471ab21ee13fa15229849e7deebb
10507,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501,003cc0338235bb1862eb7b9bdd31ec9c
3940,9.375000,NaN,NaN,8.

In [55]:
#df[df.name_mask.isin(pb)]

In [56]:
df.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
15085,2009,00037fcf2b73cebbc53964ca34bacbea,Female,Masters Women 1 (40-49),24.758503,NaN,NaN,17.013514,11.695513,NaN,22.948148,NaN,9.698333,NaN,13.434524,NaN,19.037037,NaN,NaN,16.828042,NaN,NaN,25.044025,NaN,NaN,15.080556,NaN,23.573593,NaN,14.328283,NaN,NaN,11.797297,NaN,1530.166667,17.325951
17920,2007,000601620b7a27bf6519506df6a09c6c,Male,Masters Men 1 (40-49),13.292517,NaN,NaN,8.265766,8.003205,NaN,15.240741,47.725225,NaN,NaN,NaN,NaN,15.533670,NaN,NaN,10.198413,NaN,NaN,NaN,NaN,NaN,19.630556,NaN,NaN,8.521164,NaN,NaN,8.739464,NaN,NaN,878.616667,15.515072
8449,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015
18728,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566
8339,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,NaN,17.050314,NaN,NaN,6.077778,NaN,NaN,14.325397,NaN,NaN,9.009579,NaN,NaN,887.266667,10.436679


In [57]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = df.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [58]:
#merge = merge.groupby('name_mask').head(1)

In [59]:
merge.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2009,00037fcf2b73cebbc53964ca34bacbea,Female,Masters Women 1 (40-49),24.758503,NaN,NaN,17.013514,11.695513,NaN,22.948148,NaN,9.698333,NaN,13.434524,NaN,19.037037,NaN,NaN,16.828042,NaN,NaN,25.044025,NaN,NaN,15.080556,NaN,23.573593,NaN,14.328283,NaN,NaN,11.797297,NaN,1530.166667,17.325951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007,000601620b7a27bf6519506df6a09c6c,Male,Masters Men 1 (40-49),13.292517,NaN,NaN,8.265766,8.003205,NaN,15.240741,47.725225,NaN,NaN,NaN,NaN,15.533670,NaN,NaN,10.198413,NaN,NaN,NaN,NaN,NaN,19.630556,NaN,NaN,8.521164,NaN,NaN,8.739464,NaN,NaN,878.616667,15.515072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
3,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
4,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,

In [60]:
merge = merge.dropna(subset=['time_pb'])

In [61]:
merge.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
2,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
3,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
4,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,NaN,17.050314,NaN,NaN,6.077778,NaN,NaN,14.325397,NaN,NaN,9.009579,NaN,NaN,887.266667,10.436679,7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
27,2007,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),12.030612,NaN,NaN,8.445946,8.253205,NaN,17.918519,55.873874,NaN,NaN,NaN,NaN,15.240741,NaN,NaN,10.634921,NaN,NaN,NaN,NaN,NaN,26.361111,NaN,NaN,13.309524,NaN,NaN,12.264368,NaN,NaN,1018.333333,18.033282,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,Na

In [62]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [63]:
comb

,category,gender
2,Masters Men 1 (40-49),Male
3,Senior Men (23-39),Male
4,Senior Men (23-39),Male
27,Senior Men (23-39),Male
28,Senior Men (23-39),Male
30,Masters Men 1 (40-49),Male
34,Senior Men (23-39),Male
36,Masters Men 1 (40-49),Male
37,Masters Men 1 (40-49),Male
39,Senior Men (23-39),Male


In [64]:
merge = merge.reset_index(drop=True)

In [65]:
merge.shape

(2806, 78)

In [66]:
# Update pb and ccc to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
ccc = merge.loc[:, ~merge.columns.str.contains('pb')]

In [67]:
ccc.head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
0,2012,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),9.132653,NaN,NaN,7.905405,8.506410,NaN,17.240741,NaN,7.305000,NaN,9.472619,NaN,11.508418,NaN,NaN,10.910053,NaN,NaN,19.050314,NaN,NaN,10.291667,NaN,NaN,11.460317,NaN,NaN,12.168582,NaN,NaN,954.183333,11.246015
1,2007,000bf78327bdfa8c6298fc9a76921305,Male,Senior Men (23-39),14.163265,NaN,NaN,10.069820,11.814103,NaN,20.722222,68.450450,NaN,NaN,NaN,NaN,19.287879,NaN,NaN,14.891534,NaN,NaN,NaN,NaN,NaN,33.044444,NaN,NaN,13.317460,NaN,NaN,13.534483,NaN,NaN,1236.550000,21.929566
2,2012,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),9.578231,NaN,NaN,8.236486,7.836538,NaN,15.859259,NaN,6.748333,NaN,9.576190,NaN,10.643098,NaN,NaN,10.298942,NaN,NaN,17.050314,NaN,NaN,6.077778,NaN,NaN,14.325397,NaN,NaN,9.009579,NaN,NaN,887.266667,10.436679
3,2007,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),12.030612,NaN,NaN,8.445946,8.253205,NaN,17.918519,55.873874,NaN,NaN,NaN,NaN,15.240741,NaN,NaN,10.634921,NaN,NaN,NaN,NaN,NaN,26.361111,NaN,NaN,13.309524,NaN,NaN,12.264368,NaN,NaN,1018.333333,18.033282
4,2007,00791e76e3cd48d47e4dc93d7b60ef95,Male,Senior Men (23-39),14.455782,NaN,NaN,9.018018,8.737179,NaN,15.574074,53.292793,NaN,NaN,NaN,NaN,15.873737,NaN,NaN,10.748677,NaN,NaN,NaN,NaN,NaN,25.047222,NaN,NaN,10.939153,NaN,NaN,9.818008,NaN,NaN,979.750000,17.350464


In [68]:
pb.head()

,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
1,7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
2,7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
3,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.83908,NaN,NaN,2370.466667,1133.750000,13.945114
4,8.310049,NaN,NaN,6.526316,NaN,9.925566,10.002058,15.633333,8.653595,13.300971,8.272109,17.025000,9.625926,11.991870,NaN,NaN,24.132653,10.704955,12.419872,17.788889,10.191667,11.866667,16.170034,NaN,NaN,14.177249,NaN,NaN,19.396226,NaN,9.777778,NaN,16.153680,NaN,NaN,13.343434,NaN,NaN,8.702703,2067.933333,832.983333,12.670525


# Define Functions

In [69]:
def find_similar_times(index):
    query = ccc.iloc[index]
    same_category = ccc[ccc.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(680).head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace
14,2014,00de48e074c583d8892f45726b6e6386,Male,Masters Men 1 (40-49),NaN,13.112179,8.383721,8.880631,9.378205,NaN,18.040741,NaN,7.560000,NaN,9.420238,NaN,NaN,NaN,13.761062,NaN,NaN,9.438776,18.610063,NaN,NaN,9.313889,NaN,18.333333,NaN,14.318182,NaN,NaN,8.880631,NaN,1185.966667,11.959404
35,2013,02f14ba28406d4c2310269fb61b135b3,Male,Masters Men 1 (40-49),NaN,13.338141,9.046512,8.756757,10.073718,NaN,16.074074,NaN,7.076667,NaN,9.158333,NaN,13.324916,NaN,NaN,11.301587,NaN,NaN,16.930818,NaN,NaN,9.330556,NaN,19.329004,NaN,12.772727,NaN,NaN,9.731982,NaN,1177.950000,11.874699
65,2012,06bd69d899248a71f190a37af7e3bdf5,Male,Masters Men 1 (40-49),12.234694,NaN,NaN,9.024775,9.076923,NaN,18.681481,NaN,7.486667,NaN,9.101190,NaN,14.006734,NaN,NaN,12.100529,NaN,NaN,19.383648,NaN,NaN,10.097222,NaN,NaN,10.28836,NaN,NaN,10.06705,NaN,NaN,990.900000,11.795773
84,2013,093fdd36a1a125c470a25db1619ca2cc,Male,Masters Men 1 (40-49),NaN,12.525641,9.162791,9.313063,10.000000,NaN,18.725926,NaN,7.940000,NaN,10.296429,NaN,14.744108,NaN,NaN,11.423280,NaN,NaN,16.949686,NaN,NaN,9.088889,NaN,18.562771,NaN,11.853535,NaN,NaN,9.461712,NaN,1212.766667,12.146274
87,2008,0999c52336f41c6ee35be9fb5f01fbbd,Male,Masters Men 1 (40-49),19.710884,NaN,NaN,13.216216,8.528846,NaN,15.585185,NaN,6.866667,NaN,8.359524,NaN,13.213805,NaN,NaN,10.042328,NaN,NaN,15.150943,NaN,NaN,8.430556,NaN,15.426407,NaN,10.181818,NaN,NaN,7.905405,NaN,1030.416667,11.739891


In [70]:
def calculate_euclidean_distance(index):
    similar_times = find_similar_times(index)
    similar_times = similar_times.drop(['category', 'name_mask', 'Year'], axis=1)
    similar_times = similar_times.fillna(0)
    euclidean_distance = pd.DataFrame(squareform(pdist(similar_times.iloc[:, 1:])), columns=similar_times.index.unique(), index=similar_times.index.unique())
    # Remove query runner from list of neighbours
    euclidean_distance = euclidean_distance.drop([index], axis = 0)
    euclidean_distance = euclidean_distance.loc[:,index].sort_values(ascending = True)
    euclidean_distance = euclidean_distance[euclidean_distance < euclidean_distance_threshold]
    return euclidean_distance

# Show sample output
calculate_euclidean_distance(680).head()

241      3.926860
1038     5.680651
2646     6.905713
142      6.987068
1302    10.628160
Name: 680, dtype: float64

In [71]:
def find_nearest_neighbours(index):
    euclidean_distance = calculate_euclidean_distance(index)  
    #should this be non pb time???????
    nearest_neighbours = merge.iloc[euclidean_distance.index]
    #nearest_neighbours = ccc.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(680).head()

,Year,name_mask,gender,category,Courmayeur to Bertone,Courmayeur to Tronche,Tronche to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Bonatti to Col Ferret,Arnouvaz to Col Ferret,Col Ferret to Champex La,Col Ferret to La Fouly,La Fouly to Praz de Fort,La Fouly to Champex La,Praz de Fort to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Jeurs,Trient to Vallorcine,La Catogne to Vallorcine,Jeurs to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Tete aux Vents to Flegere,Argentiere to Tines,Argentiere to Chamonix,Flegere to Chamonix,Tines to Chamonix,time,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
241,2009,14ce417ddb5a7730226a90a2826fc036,Male,Masters Men 1 (40-49),18.897959,NaN,NaN,11.957207,8.240385,NaN,17.196296,NaN,6.601667,NaN,9.163095,NaN,13.144781,NaN,NaN,9.679894,NaN,NaN,16.191824,NaN,NaN,7.319444,NaN,18.261905,NaN,10.818182,NaN,NaN,9.096847,NaN,1060.100000,12.043807,8.436275,NaN,NaN,6.697368,NaN,9.234628,12.104938,15.684848,5.947712,12.723301,6.234694,16.595833,7.496296,8.467480,NaN,NaN,23.374150,10.788288,10.945513,20.122222,9.153333,11.985714,NaN,17.836983,NaN,NaN,22.777778,NaN,20.971698,NaN,11.152778,NaN,NaN,11.928571,NaN,NaN,15.60728,NaN,NaN,2184.033333,947.316667,12.881204
1038,2009,5c79ccf6b9e3a75ef6c69d87e3753751,Male,Masters Men 1 (40-49),17.017007,NaN,NaN,11.831081,7.282051,NaN,14.759259,NaN,6.546667,NaN,8.226190,NaN,12.804714,NaN,NaN,9.312169,NaN,NaN,16.965409,NaN,NaN,9.538889,NaN,19.000000,NaN,12.994949,NaN,NaN,10.882883,NaN,1058.150000,12.089328,8.839461,NaN,NaN,7.008772,NaN,9.409385,14.277778,17.939394,9.983660,15.893204,9.857143,18.995833,10.262963,12.317073,NaN,NaN,30.040816,13.729730,13.663462,33.885185,11.706667,14.339286,NaN,NaN,22.165192,NaN,NaN,13.755102,36.729560,NaN,11.733333,NaN,24.123377,NaN,NaN,15.808081,NaN,NaN,9.981982,2605.166667,1393.433333,16.101935
2646,2009,eed3f48919141b3a42a1ee7e81ae84b8,Male,Masters Men 1 (40-49),19.755102,NaN,NaN,12.729730,8.653846,NaN,16.970370,NaN,6.528333,NaN,8.438095,NaN,12.744108,NaN,NaN,9.412698,NaN,NaN,16.531447,NaN,NaN,9.022222,NaN,17.528139,NaN,11.964646,NaN,NaN,9.162162,NaN,1065.250000,12.264685,8.817402,NaN,NaN,7.629386,NaN,8.736246,11.534979,16.715152,8.522876,13.504854,7.945578,15.550000,10.577778,10.410569,NaN,NaN,30.292517,11.536036,11.317308,20.448148,13.186667,15.046429,NaN,23.677616,NaN,NaN,18.664530,NaN,25.801887,NaN,14.186111,NaN,NaN,14.793651,NaN,NaN,16.13410,NaN,NaN,2479.816667,1243.100000,14.566514
142,2009,0d67b4bdefc7f0a77e3f9edf05a4b20a,Male,Masters Men 1 (40-49),19.272109,NaN,NaN,13.364865,8.583333,NaN,16.925926,NaN,7.058333,NaN,8.297619,NaN,11.791246,NaN,NaN,9.701058,NaN,NaN,16.993711,NaN,NaN,9.036111,NaN,17.538961,NaN,11.919192,NaN,NaN,9.168919,NaN,1065.250000,12.280876,8.127451,NaN,NaN,6.513158,NaN,10.333333,10.720165,16.906061,9.869281,12.948220,9.928571,20.404167,11.303704,13.621951,NaN,NaN,27.346939,11.459459,10.689103,22.137037,9.380000,11

In [72]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(680)
get_mean_pb(neighbours)

15.007461777657896

In [73]:
def get_mean_pace(neighbours):
    return neighbours.mean()
  
# Show sample output    
#neighbours = find_nearest_neighbours(680)
#neighbours = neighbours.filter(like='pb')
#get_mean_pace(neighbours)

In [74]:
def get_best_ccc(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(680)
get_best_ccc(neighbours)

11.604050156669901

In [75]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(680)
get_best_pb(neighbours)

10.422477570172058

In [76]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


7.350387596899226

In [77]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_ccc(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(680)
get_prediction(680, neighbours)

10.857534506300121

# Calculate Predictions

In [78]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_prediction'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_ccc(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    return row

In [79]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_prediction':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_prediction']]
df

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_prediction
0,11.246015,15.222938,11.155820,14.202531
1,21.929566,15.222938,13.963810,16.132048
2,10.436679,12.416371,10.200935,13.011486
3,18.033282,13.945114,12.960100,15.386507
4,17.350464,12.670525,13.933041,15.058385
5,11.414342,15.628671,10.890606,14.293687
6,17.027937,16.952247,14.267979,16.178266
7,10.836450,14.358531,11.163049,13.909187
8,11.295707,14.358531,11.143323,14.276569
9,10.192421,16.232838,10.150220,12.617810


In [80]:
df = comb.join(df)

In [81]:
df.to_csv('../../data/CCC_predictions.csv')

# Mean Pace Recommendations

In [82]:
def update_pace(line):
    index = int(line['Chamonix to Delevret_pb'])
    neighbours = find_nearest_neighbours(index)
    line['Chamonix to St-Gervais_pb'] = Cham_to_StG(neighbours)
    line['Chamonix to La Charme_pb'] = Cham_to_LaCh(neighbours)
    line['Delevret to St-Gervais_pb'] = Del_to_StG(neighbours)
    line['La Charme to St-Gervais_pb'] = LaCh_to_StG(neighbours)
    line['St-Gervais to Contamines_pb'] = StG_to_Cont(neighbours)
    line['Contamines to La Balme_pb'] = Cont_to_LaB(neighbours)
    line['La Balme to Bonhomme_pb'] = LaB_to_Bon(neighbours)
    line['Bonhomme to Chapieux_pb'] = Bon_to_Chap(neighbours)
    line['Chapieux to Col Seigne_pb'] = Chap_to_ColS(neighbours)
    line['Col Seigne to Lac Combal_pb'] = ColS_to_Lac(neighbours)
    line['Lac Combal to Mt-Favre_pb'] = Lac_to_MtF(neighbours)
    line['Mt-Favre to Checruit_pb '] = MtF_to_Chec(neighbours)
    line['Checruit to Courmayeur_pb'] = Chec_to_Courm(neighbours)
    line['Courmayeur to Courmayeur2_pb'] = Courm_to_Courm2(neighbours)
    line['Courmayeur2 to Bertone_pb'] = Courm2_to_Bertone(neighbours)
    line['Courmayeur to Bertone_pb'] = Courm_to_Bertone(neighbours)
    line['Bertone to Bonatti_pb'] = Bertone_to_Bon(neighbours)
    line['Bonatti to Arnouvaz_pb'] = Bon_to_Arn(neighbours)
    line['Arnouvaz to Col Ferret_pb'] = Arn_to_Col(neighbours)
    line['Col Ferret to La Fouly_pb'] = Col_to_LaF(neighbours)
    line['La Fouly to Champex La_pb'] = LaF_to_Champ(neighbours)
    line['Champex La to Bovine_pb'] = Champ_to_Bov(neighbours)
    line['Champex La to Mortigny_pb'] = Champ_to_Mort(neighbours)
    line['Champex La to Giete_pb'] = Champ_to_Giete(neighbours)
    line['Bovine to Trient_pb'] = Bov_to_Trient(neighbours)
    line['Mortigny to Trient_pb'] = Mort_to_Trient(neighbours)
    line['Giete to Trient_pb'] = Giete_to_Trient(neighbours)
    line['Trient to La Catogne_pb'] = Trient_to_LaCa(neighbours)
    line['Trient to Les Tseppe_pb'] = Trient_to_LesSt(neighbours)
    line['La Catogne to Vallorcine_pb'] = LaCa_to_Vall(neighbours)
    line['Les Tseppe to Vallorcine_pb'] = LesSt_to_Vall(neighbours)
    line['Vallorcine to Tete aux Vents_pb'] = Vall_to_tAV(neighbours)
    line['Vallorcine to Argentiere_pb'] = Vall_to_Arg(neighbours)
    line['Vallorcine to Col Montet_pb'] = Vall_to_Col(neighbours)
    line['Tete aux Vents to Flegere_pb'] = tAV_to_Fleg(neighbours)
    line['Argentiere to Chamonix_pb'] = Arg_to_Cham(neighbours)
    line['Col Montet to Flegere_pb'] = Col_to_Fleg(neighbours)
    line['Flegere to Chamonix_pb'] = Fleg_to_Cham(neighbours)
    
    line['Chamonix to Delevret_pb'] = Cham_to_Del(neighbours)
    
    return line

In [83]:
num_of_runs1 = merge.time.count()
filler_data = np.arange(0.0, num_of_runs1)
df1 = pd.DataFrame(
    { 
     'Chamonix to Delevret_pb':filler_data,            
'Chamonix to La Charme_pb':filler_data,            
     'Chamonix to St-Gervais_pb':filler_data,      
'Delevret to St-Gervais_pb':filler_data,           
'La Charme to St-Gervais_pb':filler_data,          
'St-Gervais to Contamines_pb':filler_data,         
'Contamines to La Balme_pb':filler_data,           
'La Balme to Bonhomme_pb':filler_data,             
     'Bonhomme to Chapieux_pb':filler_data,              
'Chapieux to Col Seigne_pb':filler_data,           
'Col Seigne to Lac Combal_pb':filler_data,         
'Lac Combal to Mt-Favre_pb':filler_data,           
'Mt-Favre to Checruit_pb':filler_data,             
'Checruit to Courmayeur_pb':filler_data,           
'Courmayeur to Courmayeur2_pb':filler_data,        
'Courmayeur2 to Bertone_pb':filler_data,           
'Courmayeur to Bertone_pb':filler_data,           
'Bertone to Bonatti_pb':filler_data,               
'Bonatti to Arnouvaz_pb':filler_data,              
'Arnouvaz to Col Ferret_pb':filler_data,           
'Col Ferret to La Fouly_pb':filler_data,           
'La Fouly to Champex La_pb':filler_data,           
'Champex La to Bovine_pb':filler_data,             
'Champex La to Mortigny_pb':filler_data,           
'Champex La to Giete_pb':filler_data,             
'Bovine to Trient_pb':filler_data,                 
'Mortigny to Trient_pb':filler_data,               
'Giete to Trient_pb':filler_data,                  
'Trient to La Catogne_pb':filler_data,             
'Trient to Les Tseppe_pb':filler_data,             
'La Catogne to Vallorcine_pb':filler_data,         
'Les Tseppe to Vallorcine_pb':filler_data,         
'Vallorcine to Tete aux Vents_pb':filler_data,     
'Vallorcine to Argentiere_pb':filler_data,         
'Vallorcine to Col Montet_pb':filler_data,         
'Tete aux Vents to Flegere_pb':filler_data,        
'Argentiere to Chamonix_pb':filler_data,           
'Col Montet to Flegere_pb':filler_data,            
'Flegere to Chamonix_pb':filler_data             
    }
)

df1 = df1.apply(lambda line: update_pace(line), axis=1)
df1 = df1[['Chamonix to Delevret_pb',            
'Chamonix to La Charme_pb',            
     'Chamonix to St-Gervais_pb',      
'Delevret to St-Gervais_pb',           
'La Charme to St-Gervais_pb',          
'St-Gervais to Contamines_pb',         
'Contamines to La Balme_pb',           
'La Balme to Bonhomme_pb',             
     'Bonhomme to Chapieux_pb',              
'Chapieux to Col Seigne_pb',           
'Col Seigne to Lac Combal_pb',         
'Lac Combal to Mt-Favre_pb',           
'Mt-Favre to Checruit_pb',             
'Checruit to Courmayeur_pb',           
'Courmayeur to Courmayeur2_pb',        
'Courmayeur2 to Bertone_pb',           
'Courmayeur to Bertone_pb',           
'Bertone to Bonatti_pb',               
'Bonatti to Arnouvaz_pb',              
'Arnouvaz to Col Ferret_pb',           
'Col Ferret to La Fouly_pb',           
'La Fouly to Champex La_pb',           
'Champex La to Bovine_pb',             
'Champex La to Mortigny_pb',           
'Champex La to Giete_pb',             
'Bovine to Trient_pb',                 
'Mortigny to Trient_pb',               
'Giete to Trient_pb',                  
'Trient to La Catogne_pb',             
'Trient to Les Tseppe_pb',             
'La Catogne to Vallorcine_pb',         
'Les Tseppe to Vallorcine_pb',         
'Vallorcine to Tete aux Vents_pb',     
'Vallorcine to Argentiere_pb',         
'Vallorcine to Col Montet_pb',         
'Tete aux Vents to Flegere_pb',        
'Argentiere to Chamonix_pb',           
'Col Montet to Flegere_pb',            
'Flegere to Chamonix_pb']]
df1.head()

,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,8.294701,4.709524,7.350388,7.121259,15.099691,9.441002,11.932262,16.867961,9.280475,13.845273,11.601645,17.688552,0.0,10.695990,43.761243,22.384028,24.076895,11.541000,12.298454,21.688493,9.604272,12.173521,16.579461,15.004380,18.225359,13.924036,19.536325,12.956430,22.517408,19.932383,11.974471,18.211585,21.716176,13.215873,20.573969,15.840174,12.611494,19.003759,11.156141
1,8.837010,6.502319,9.534574,7.155702,22.106481,10.779011,14.426955,18.883983,12.173203,15.621128,10.182459,19.311607,1.0,12.728513,NaN,NaN,28.532556,14.012387,13.482143,24.256878,12.500357,16.367007,23.703574,19.229927,NaN,17.651404,21.504274,NaN,23.507637,NaN,12.513095,NaN,22.878122,14.791005,NaN,13.763403,14.180077,NaN,11.392585
2,7.755988,4.195411,7.355592,6.322621,16.811667,8.738006,10.927350,15.524975,8.472025,12.810039,10.298198,16.038426,2.0,9.859108,40.837778,20.214063,21.774286,10.539526,10.887257,19.578958,8.957949,11.210060,15.945057,14.093674,16.132416,14.043691,17.067842,10.678634,20.551908,18.033420,11.080148,15.067529,19.735473,11.287698,19.554054,14.859866,11.049808,18.418006,10.682432
3,8.987045,NaN,8.244186,7.907268,35.043056,10.163906,13.130840,17.719964,10.392926,14.750999,9.470588,17.909314,3.0,11.776901,NaN,NaN,28.020608,12.676603,13.633296,22.894553,12.009020,15.443697,20.598345,18.485706,16.067847,15.970238,18.311966,14.853741,23.786349,NaN,12.807026,NaN,21.708625,11.360450,NaN,15.598679,11.054119,NaN,13.236313
4,NaN,5.201449,8.275039,NaN,15.550000,10.453398,13.172634,17.694545,10.647059,14.593366,9.634354,17.632917,4.0,12.490650,NaN,NaN,25.449660,12.970270,12.913141,22.082593,10.941667,13.290238,20.031313,NaN,NaN,15.953704,NaN,NaN,24.521698,NaN,15.085000,NaN,20.713636,NaN,NaN,14.440909,NaN,NaN,11.967568


# Best Pace Recommendations

In [84]:
def Cham_2_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].min()

def Cham_2_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].min()

def Cham_2_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].min()

def Del_2_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].min()

def LaCh_2_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].min()

def StG_2_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].min()

def Cont_2_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].min()

def LaB_2_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].min()

def Bon_2_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].min()

def Chap_2_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].min()

def ColS_2_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].min()

def Lac_2_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].min()

def MtF_2_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].min()

def Chec_2_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].min()

def Courm_2_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].min()

def Courm2_2_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].min()

def Courm_2_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].min()

def Bertone_2_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].min()

def Bon_2_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].min()

def Arn_2_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].min()

def Col_2_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].min()

def LaF_2_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].min()

def Champ_2_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].min()

def Champ_2_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].min()

def Champ_2_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].min()

def Bov_2_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].min()

def Mort_2_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].min()

def Giete_2_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].min()

def Trient_2_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].min()

def Trient_2_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].min()

def LaCa_2_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].min()

def LesSt_2_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].min()

def Vall_2_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].min()

def Vall_2_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].min()

def Vall_2_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].min()

def tAV_2_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].min()

def Arg_2_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].min()

def Col_2_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].min()

def Fleg_2_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].min()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_2_StG(neighbours)
#Cham_2_LaCh(neighbours)
#Cham_2_StG(neighbours)   


5.7651162790697672

In [85]:
def update_best(sta):
    index = int(sta['Chamonix to Delevret_pb'])
    neighbours = find_nearest_neighbours(index)
    sta['Chamonix to St-Gervais_pb'] = Cham_2_StG(neighbours)
    sta['Chamonix to La Charme_pb'] = Cham_2_LaCh(neighbours)
    sta['Delevret to St-Gervais_pb'] = Del_2_StG(neighbours)
    sta['La Charme to St-Gervais_pb'] = LaCh_2_StG(neighbours)
    sta['St-Gervais to Contamines_pb'] = StG_2_Cont(neighbours)
    sta['Contamines to La Balme_pb'] = Cont_2_LaB(neighbours)
    sta['La Balme to Bonhomme_pb'] = LaB_2_Bon(neighbours)
    sta['Bonhomme to Chapieux_pb'] = Bon_2_Chap(neighbours)
    sta['Chapieux to Col Seigne_pb'] = Chap_2_ColS(neighbours)
    sta['Col Seigne to Lac Combal_pb'] = ColS_2_Lac(neighbours)
    sta['Lac Combal to Mt-Favre_pb'] = Lac_2_MtF(neighbours)
    sta['Mt-Favre to Checruit_pb '] = MtF_2_Chec(neighbours)
    sta['Checruit to Courmayeur_pb'] = Chec_2_Courm(neighbours)
    sta['Courmayeur to Courmayeur2_pb'] = Courm_2_Courm2(neighbours)
    sta['Courmayeur2 to Bertone_pb'] = Courm2_2_Bertone(neighbours)
    sta['Courmayeur to Bertone_pb'] = Courm_2_Bertone(neighbours)
    sta['Bertone to Bonatti_pb'] = Bertone_2_Bon(neighbours)
    sta['Bonatti to Arnouvaz_pb'] = Bon_2_Arn(neighbours)
    sta['Arnouvaz to Col Ferret_pb'] = Arn_2_Col(neighbours)
    sta['Col Ferret to La Fouly_pb'] = Col_2_LaF(neighbours)
    sta['La Fouly to Champex La_pb'] = LaF_2_Champ(neighbours)
    sta['Champex La to Bovine_pb'] = Champ_2_Bov(neighbours)
    sta['Champex La to Mortigny_pb'] = Champ_2_Mort(neighbours)
    sta['Champex La to Giete_pb'] = Champ_2_Giete(neighbours)
    sta['Bovine to Trient_pb'] = Bov_2_Trient(neighbours)
    sta['Mortigny to Trient_pb'] = Mort_2_Trient(neighbours)
    sta['Giete to Trient_pb'] = Giete_2_Trient(neighbours)
    sta['Trient to La Catogne_pb'] = Trient_2_LaCa(neighbours)
    sta['Trient to Les Tseppe_pb'] = Trient_2_LesSt(neighbours)
    sta['La Catogne to Vallorcine_pb'] = LaCa_2_Vall(neighbours)
    sta['Les Tseppe to Vallorcine_pb'] = LesSt_2_Vall(neighbours)
    sta['Vallorcine to Tete aux Vents_pb'] = Vall_2_tAV(neighbours)
    sta['Vallorcine to Argentiere_pb'] = Vall_2_Arg(neighbours)
    sta['Vallorcine to Col Montet_pb'] = Vall_2_Col(neighbours)
    sta['Tete aux Vents to Flegere_pb'] = tAV_2_Fleg(neighbours)
    sta['Argentiere to Chamonix_pb'] = Arg_2_Cham(neighbours)
    sta['Col Montet to Flegere_pb'] = Col_2_Fleg(neighbours)
    sta['Flegere to Chamonix_pb'] = Fleg_2_Cham(neighbours)
    
    sta['Chamonix to Delevret_pb'] = Cham_2_Del(neighbours)
    
    return sta

In [86]:
num_of_runs2 = merge.time.count()
filler_data = np.arange(0.0, num_of_runs2)
df2 = pd.DataFrame(
    { 
     'Chamonix to Delevret_pb':filler_data,            
'Chamonix to La Charme_pb':filler_data,            
     'Chamonix to St-Gervais_pb':filler_data,      
'Delevret to St-Gervais_pb':filler_data,           
'La Charme to St-Gervais_pb':filler_data,          
'St-Gervais to Contamines_pb':filler_data,         
'Contamines to La Balme_pb':filler_data,           
'La Balme to Bonhomme_pb':filler_data,             
     'Bonhomme to Chapieux_pb':filler_data,              
'Chapieux to Col Seigne_pb':filler_data,           
'Col Seigne to Lac Combal_pb':filler_data,         
'Lac Combal to Mt-Favre_pb':filler_data,           
'Mt-Favre to Checruit_pb':filler_data,             
'Checruit to Courmayeur_pb':filler_data,           
'Courmayeur to Courmayeur2_pb':filler_data,        
'Courmayeur2 to Bertone_pb':filler_data,           
'Courmayeur to Bertone_pb':filler_data,           
'Bertone to Bonatti_pb':filler_data,               
'Bonatti to Arnouvaz_pb':filler_data,              
'Arnouvaz to Col Ferret_pb':filler_data,           
'Col Ferret to La Fouly_pb':filler_data,           
'La Fouly to Champex La_pb':filler_data,           
'Champex La to Bovine_pb':filler_data,             
'Champex La to Mortigny_pb':filler_data,           
'Champex La to Giete_pb':filler_data,             
'Bovine to Trient_pb':filler_data,                 
'Mortigny to Trient_pb':filler_data,               
'Giete to Trient_pb':filler_data,                  
'Trient to La Catogne_pb':filler_data,             
'Trient to Les Tseppe_pb':filler_data,             
'La Catogne to Vallorcine_pb':filler_data,         
'Les Tseppe to Vallorcine_pb':filler_data,         
'Vallorcine to Tete aux Vents_pb':filler_data,     
'Vallorcine to Argentiere_pb':filler_data,         
'Vallorcine to Col Montet_pb':filler_data,         
'Tete aux Vents to Flegere_pb':filler_data,        
'Argentiere to Chamonix_pb':filler_data,           
'Col Montet to Flegere_pb':filler_data,            
'Flegere to Chamonix_pb':filler_data             
    }
)

df2 = df2.apply(lambda sta: update_best(sta), axis=1)
df2 = df2[['Chamonix to Delevret_pb',            
'Chamonix to La Charme_pb',            
     'Chamonix to St-Gervais_pb',      
'Delevret to St-Gervais_pb',           
'La Charme to St-Gervais_pb',          
'St-Gervais to Contamines_pb',         
'Contamines to La Balme_pb',           
'La Balme to Bonhomme_pb',             
     'Bonhomme to Chapieux_pb',              
'Chapieux to Col Seigne_pb',           
'Col Seigne to Lac Combal_pb',         
'Lac Combal to Mt-Favre_pb',           
'Mt-Favre to Checruit_pb',             
'Checruit to Courmayeur_pb',           
'Courmayeur to Courmayeur2_pb',        
'Courmayeur2 to Bertone_pb',           
'Courmayeur to Bertone_pb',           
'Bertone to Bonatti_pb',               
'Bonatti to Arnouvaz_pb',              
'Arnouvaz to Col Ferret_pb',           
'Col Ferret to La Fouly_pb',           
'La Fouly to Champex La_pb',           
'Champex La to Bovine_pb',             
'Champex La to Mortigny_pb',           
'Champex La to Giete_pb',             
'Bovine to Trient_pb',                 
'Mortigny to Trient_pb',               
'Giete to Trient_pb',                  
'Trient to La Catogne_pb',             
'Trient to Les Tseppe_pb',             
'La Catogne to Vallorcine_pb',         
'Les Tseppe to Vallorcine_pb',         
'Vallorcine to Tete aux Vents_pb',     
'Vallorcine to Argentiere_pb',         
'Vallorcine to Col Montet_pb',         
'Tete aux Vents to Flegere_pb',        
'Argentiere to Chamonix_pb',           
'Col Montet to Flegere_pb',            
'Flegere to Chamonix_pb']]
df2.head()

,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,6.837010,3.739130,5.765116,5.517544,6.280556,7.059871,8.720165,13.006061,7.062092,10.804207,6.102041,13.558333,0.0,8.247967,4.148148,16.5250,14.993197,9.038288,8.897436,15.981481,6.698333,8.890476,12.654882,12.826034,12.901180,9.365079,16.081197,7.387755,16.207547,15.666667,6.205556,13.658046,16.056277,9.674603,12.256757,8.515152,8.551724,15.076190,6.855856
1,8.837010,4.497101,8.427907,7.155702,8.427778,8.071197,10.187243,15.315152,7.343137,12.773463,6.595238,16.483333,1.0,8.869919,NaN,NaN,23.200680,10.065315,9.910256,19.614815,9.535000,12.845238,16.597643,19.229927,NaN,14.301587,21.504274,NaN,18.952830,NaN,10.530556,NaN,16.439394,14.791005,NaN,9.787879,14.180077,NaN,7.297297
2,5.895833,4.002174,6.178295,4.844298,6.158333,6.569579,7.798354,12.181818,6.506536,10.155340,5.289116,12.066667,2.0,3.495935,14.407407,16.8625,13.799320,8.105856,7.564103,14.125926,6.106667,7.727381,10.823232,10.553528,11.930678,9.288360,14.923077,7.170068,13.808176,14.510417,6.305556,11.123563,13.703463,7.859788,12.878378,6.954545,8.166667,13.957143,5.135135
3,8.307598,NaN,7.477519,6.682018,34.230556,8.687702,10.366255,14.881818,7.928105,12.509709,7.649660,13.100000,3.0,7.398374,NaN,NaN,21.013605,9.353604,9.830128,18.951852,8.846667,10.896429,14.498316,16.442822,16.067847,12.489418,17.314103,14.853741,18.474843,NaN,8.280556,NaN,17.307359,8.645503,NaN,11.500000,7.683908,NaN,8.376126
4,NaN,4.741304,7.477519,NaN,8.180556,8.423948,12.164609,16.251515,8.666667,12.915858,8.537415,15.737500,4.0,9.873984,NaN,NaN,20.244898,11.995495,11.259615,19.407407,10.191667,10.672619,14.634680,NaN,NaN,13.854497,NaN,NaN,21.261006,NaN,10.319444,NaN,16.666667,NaN,NaN,11.500000,NaN,NaN,8.376126
